# SFEs

In [1]:
# set up force field as per usual

from openff.evaluator.forcefield import SmirnoffForceFieldSource

force_field_path = "openff-2.0.0.offxml"
force_field_source = SmirnoffForceFieldSource.from_path(force_field_path)

In [2]:
# set up dataset

from openff.evaluator.datasets import PhysicalPropertyDataSet
dataset = PhysicalPropertyDataSet.from_json("freesolv.json")

/home/bamo6610/miniconda3/envs/openff-sage/lib/python3.9/site-packages/cmiles/_cmiles_rd.py:349: UserWarning: Stereochemistry is unspecified. Problematic atoms [(4, 'C'), (5, 'C'), (8, 'C'), (14, 'C')], problematic bonds []
  warnings.warn("Stereochemistry is unspecified. Problematic atoms {}, problematic bonds {}".format(


In [3]:
dataset._properties = dataset.properties[:1] # shrink to 1 property

In [4]:
dataset.properties[0]

In [6]:
dataset.properties_by_substance

<bound method PhysicalPropertyDataSet.properties_by_substance of <PhysicalPropertyDataSet n_properties=1 n_substances=1 n_sources=1>>

In [5]:
# set up dask local cluster etc

import os

from openff.evaluator.backends import ComputeResources
from openff.evaluator.backends.dask import DaskLocalCluster

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

calculation_backend = DaskLocalCluster(
    number_of_workers=1,
    resources_per_worker=ComputeResources(
        number_of_threads=1,
        number_of_gpus=1,
        preferred_gpu_toolkit=ComputeResources.GPUToolkit.CUDA,
    ),
)
calculation_backend.start()


In [6]:
# set up server

from openff.evaluator.server import EvaluatorServer

evaluator_server = EvaluatorServer(calculation_backend=calculation_backend)
evaluator_server.start(asynchronous=True)

In [7]:
from openff.evaluator.client import EvaluatorClient

evaluator_client = EvaluatorClient()

request, exception = evaluator_client.request_estimate(
    property_set=dataset,
    force_field_source=force_field_source,
    # options=estimation_options,
)

assert exception is None

/Users/lily/micromamba/envs/openff-sage/lib/python3.9/site-packages/openff/units/openmm.py:13: UserWarning: Found units module in simtk namespace, not openmm. Use openff.units.simtk instead.
  warnings.warn(
/Users/lily/micromamba/envs/openff-sage/lib/python3.9/site-packages/openff/units/openmm.py:13: UserWarning: Found units module in simtk namespace, not openmm. Use openff.units.simtk instead.
  warnings.warn(
/Users/lily/micromamba/envs/openff-sage/lib/python3.9/site-packages/openff/units/openmm.py:13: UserWarning: Found units module in simtk namespace, not openmm. Use openff.units.simtk instead.
  warnings.warn(
/Users/lily/micromamba/envs/openff-sage/lib/python3.9/site-packages/openff/units/openmm.py:13: UserWarning: Found units module in simtk namespace, not openmm. Use openff.units.simtk instead.
  warnings.warn(
/Users/lily/micromamba/envs/openff-sage/lib/python3.9/site-packages/openff/units/openmm.py:13: UserWarning: Found units module in simtk namespace, not openmm. Use openf

In [8]:
results, exception = request.results(synchronous=True, polling_interval=30)
assert exception is None